In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
train = pd.read_csv('strain0.csv')
test = pd.read_csv('stest0.csv')
result=pd.read_csv('sample_submission_2.csv')
del train['Unnamed: 0']
del test['Unnamed: 0']
print (train.shape)
print (test.shape)

(40000, 213)
(120000, 149)


In [2]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
def extrare(xtrain1,ytrain1,xtrain,ytrain,X_test,weight):
    parameter_grid={'n_estimators' : range(20,81,10),
                    'max_depth':(15,18,20),
                'min_samples_split':  range(2,10,2),
                'min_samples_leaf':  range(1, 10)
                }
    grid_extra = GridSearchCV(ExtraTreesRegressor(),
                           param_grid = parameter_grid,
                           scoring = 'mean_squared_error',
                           cv = 10)
    grid_extra.fit(xtrain1, ytrain1)
    print('Best score (ExtraTreesRegressor): {}'.format(grid_extra.best_score_))
    print('Best parameters: {}'.format(grid_extra.best_params_))
    clf = ExtraTreesRegressor(grid_extra.best_estimator_)
    clf.fit(xtrain, ytrain,weight)
    r=clf.predict(X_test)
    return r

train2=train
for i in range(0,149):
    train2.iloc[:,i]=np.multiply(train.iloc[:,i],np.sqrt(train.iloc[:,211]))
train3=train
for i in range(0,149):
    train3.iloc[:,i]=np.multiply(train.iloc[:,i],np.sqrt(train.iloc[:,212]))
    
ridpredict = np.zeros((120000, 62))
for i in range(0,62):
    if i==60:
        yt=np.multiply(train.iloc[:,149+i],np.sqrt(train.iloc[:,212]))
        ridpredict[:,i]=extrare(train3.iloc[:,0:149],yt,train.iloc[:,0:149],train.iloc[:,149+i],test,train.iloc[:,212])
    elif i==61:
        yt=np.multiply(train.iloc[:,149+i],np.sqrt(train.iloc[:,212]))
        ridpredict[:,i]=extrare(train3.iloc[:,0:149],yt,train.iloc[:,0:149],train.iloc[:,149+i],test,train.iloc[:,212])
    else:
        yt=np.multiply(train.iloc[:,149+i],np.sqrt(train.iloc[:,211]))
        ridpredict[:,i]=extrare(train2.iloc[:,0:149],yt,train.iloc[:,0:149],train.iloc[:,149+i],test,train.iloc[:,211])
        
        
resultnum=np.zeros(7440000)
for j in range(0,120000):
    resultnum[62*j:62+62*j]=ridpredict[j,:]
result.iloc[0:,1]=resultnum
result.to_csv('result-extra-wse.csv',index=False)



C:\Users\Seven\Anaconda3\envs\py27\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\Seven\Anaconda3\envs\py27\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


KeyboardInterrupt: 